## Komodo Energy Coding Challenge

In [ ]:
#!pip list
#from platform import python_version
#print(python_version())

In [ ]:
import numpy as np #numpy==1.24.3
import pandas as pd #pandas==2.0.3
import matplotlib.pyplot as plt #matplotlib==3.7.2
import datetime as dt #python 3.11.5

# 1. Data Exploration

## Household usage data

In [ ]:
# Read in the household usage data
energy = pd.read_csv('household_usage.csv')
energy = energy[['Time', 'Standard_Household', 'HeatPump_Household', 'HeatPump_Battery_Household']]
energy

In [ ]:
# Plot the timeseries
energy.set_index('Time').plot(style='.-', ylabel='kWh')


In [ ]:
# Get some useful stats
energy.set_index('Time').describe()

In [ ]:
# Find the time when the energy consumption is the max
e = energy.set_index('Time')
e[e==e.max()].dropna(how='all')

In [ ]:
# Find the time when the energy consumption is the minimum
energy.set_index('Time').idxmin()


In [ ]:
# Find the daily energy usage
energy.set_index('Time').sum()


### How do different households use energy at different times of day?

The standard household represents a typical UK home where the consumption is mainly in the evening.
The household are likely out during the day at work and return at 5.30pm.
There is a base load of 0.2kWh and a peak load of 1.5kWh at 18:30-19:00.
The total consumption is ~18kWh.

The Heat pump household has large peaks in the morning and the evening.
This is presumably when the heating/hot water are on. The maximum energy use in a half hour period was 2.2kWh.
The total consumption is approximately double the standard household at 35 kWh per day.

The Heat Pump battery household has a similar total consuption to the heat pump household but the times of usage are different. If we assume the home usage for the two homes are identical, this suggests that we're modelling a battery that is 100% efficient. That is, all the energy that goes into the battery is available for the home to use.
The battery is charged overight and early afternoon (2.30-4pm), presumably when the tariff is cheaper. I've shown in the Tariff section that this is the case.
Home usage is zero from 4am until 20:30 when the charge in the battery runs out. 
The battery charges for 90 mins at 5kWh per 30 mins. So this is probably a ~12-15kWh battery. We could do the full calculation by subtracting the heat pump only usage and suming the remaining energy used between battery charge times. In reality you would expect a battery system to use more due to inverter losses.

No missing data or outliers seen. Data is for a single day.

## Tariff data

In [ ]:
# Read in tariff data
tariff = pd.read_json('agile_price_example.json')
tariff


In [ ]:
# Extract the datetime and prices that include VAT
valid_from = [x['valid_from'] for x in tariff['results']]
valid_from = pd.to_datetime(pd.Series(valid_from))
value_inc_vat = [x['value_inc_vat'] for x in tariff['results']]
prices = pd.DataFrame(value_inc_vat, index=valid_from).sort_index()
prices.columns=['tariff']
prices

In [ ]:
prices.describe()

In [ ]:
prices.plot(ylabel='p/kWh')

We have data from Tues 11/3/25 23:00 to Thu 13/3/25 23:00. So 48 hours but not quite 2 full days. The pattern in the two days is similar but slightly different. There's a peak in the morning and a larger peak in the evening around 6pm when people use most energy. 

In [ ]:
# Overplot the data 
prices['date']=prices.index.date
prices['time']=prices.index.time
prices.set_index('time').groupby(['date'])['tariff'].plot(legend=True, ylabel='p/kWh')

The amplitude slightly differs between the two days, particularly the morning peak and the afternoon baseline. Peak price is around 48p/kWh and minimum price is around 19p/kWh on both days. NB These are both weekdays


No missing data or outliers seen. 


## Flexibility Event data

In [ ]:
# Read in flexibilty data
flex = pd.read_json('flexibility_opportunity.json')
flex

In [ ]:
flex = pd.json_normalize(flex['flexibility_opportunities'])
flex

In [ ]:
# Calculate the maximum return from each event
flex['price_per_kWh']*flex['max_flexibility_kWh']

There are two events specified - a turn down (reduce consumption) and a turn up (use excess grid energy). Turn down is more lucrative and could earn £7.50 if the maximum energy is saved (5kWh). If you could combine the two, i.e. move your energy use from the turn down time to the turn up time, a householder could earn £9. 

The minimum flexibility for a turn down event is 1 kWh, so just shifting appliances like washing machine/oven/tumble dryer may be enough to earn a little. However, hot water and space heating are more reliably shifted with less disruption to the householder as they can be controlled automatically and have larger capacity for shifting energy. 

## Load shifting potential

In [ ]:
# Merge the datasets
energy['time'] = pd.to_datetime(flex_energy['Time'], format= '%H:%M:%S').dt.time
p=prices.groupby(['date', 'time']).max().unstack(level=0).droplevel(level=0, axis=1)
flex_pot = pd.merge(energy, p, on="time")
flex_pot.columns=['Time', 'Standard_Household', 'HeatPump_Household', 'HeatPump_Battery_Household', 'time','2025-03-11', '2025-03-12', '2025-03-13']

In [ ]:
# Plot the tariff and energy use
ax = flex_pot.set_index('time').plot(ylabel='kWh', color=['C0', 'C1', 'C2', 'black', 'black', 'black'], secondary_y=['Standard_Household', 'HeatPump_Household', 'HeatPump_Battery_Household'])
ax.set_ylabel('p/kWh')
ax.set_ylim(10,60)



The plot above shows the potential for saving through load shifting.
As we expected, the times when the battery charges are when the tariff is low, and just before the evening peak. This will help the battery last as long as possible over the most expensive time.
When the price is most expensive, in the evening, both the Standard and Heat pump households have their highest consumption. 
They could save money by shifting away from this time. The heat pump household has a higher consumption so more to gain.
The heat pump household also uses a lot of energy during the morning peak. Therefore the heatpump household has the most to gain from load shifting.

In [ ]:
flex_pot.head()

## Home Energy Cost

In [ ]:
# Calculate the cost in £ assuming the usage data is from 13/3/2025
flex_pot['SH_ag_cost'] = flex_pot['Standard_Household']*flex_pot['2025-03-12']/100
flex_pot['HP_ag_cost'] = flex_pot['HeatPump_Household']*flex_pot['2025-03-12']/100
flex_pot['HPB_ag_cost'] = flex_pot['HeatPump_Battery_Household']*flex_pot['2025-03-12']/100
# Current Ofgem price cap (1 April to 30 June 2025 )
flat_tariff = 27.03
flex_pot['SH_ft_cost'] = flex_pot['Standard_Household']*flat_tariff/100
flex_pot['HP_ft_cost'] = flex_pot['HeatPump_Household']*flat_tariff/100
flex_pot['HPB_ft_cost'] = flex_pot['HeatPump_Battery_Household']*flat_tariff/100

flex_pot.head()

In [ ]:
# Plot the daily cost for each household under agile tariff
ax = flex_pot[['SH_ag_cost','HP_ag_cost','HPB_ag_cost']].sum().plot.bar(ylabel='£')
plt.xticks(rotation=0)
for c in ax.containers: ax.bar_label(c, fmt='%.1f', label_type='edge')


The plot shows that a heat pump home could save around £3 per day from using a battery to shift the load from peak to off peak times under the agile tariff.

In [ ]:
# Plot the daily cost for each household under agile tariff
ax = flex_pot[['SH_ft_cost','HP_ft_cost','HPB_ft_cost']].sum().plot.bar(ylabel='£')
plt.xticks(rotation=0)
for c in ax.containers: ax.bar_label(c, fmt='%.1f', label_type='edge')

The standard and heat pump homes would pay slightly less under the flat tariff. The heat pump homes with and without the battery have the same cost under the flat tariff (they have the same energy usage as noted above). A heat pump home with a battery moving from a flat tariff to an agile one would save about £2 per day.

## Flexibility Provider insights

A flexibility provider could make the following conclusions based on this data:
* Battery systems are already likely to be optimised for charging at low demand times and discharging at high tariff times. Therefore they may not be so suited for turn-down events. But they could be very useful for turn-up events. They may be able to react more quickly and have more capacity for energy storage than heat pump systems.
* Heat pumps offer the most turn-down potential, as they could shift their heating/hot water earlier particularly at the evening peak time, if there is enough warning. They have the capacity to turn-down more than the standard households. 
* Heat pump homes and standard homes with hot water tanks also have the potential for turn-up events as they could store some energy in the tank.


# 2. Recomending when to shift energy use 

Imagine we have a flexible load that consumes energy. Under an agile tariff, it can be cost effective to shift this load in time if the scheduled time is during a high tariff period. The larger the load, the more cost effective it is to shift it. So lets consider devices like hot water heaters and tumble dryers.

We can specify the ideal time we would like this load to start, the average power of the device and the duration it is on for. We can then calculate the cost of using the device at this time and at other times, and see which gives the best savings. However some times may less convenient than others. We can take this into consideration by defining a cost function that penalises for inconvenience. In this scenario we consider the following criteria:
* The householder doesn't want the device on at night (11pm-6am). A large cost penalty is applied.
* The further away from the scheduled time, the less desirable it is. For example, shifting by 2 hours is more desirable than 6 hours.
* Moving the load earlier than the ideal schedule is more desirable than moving it later. The penalty for shifting later is twice that of shifting earlier. 

We create a simultor to calculate the inconvenience cost as well as monetary gain of shifting a load. The time with the least inconvenience for the most gain is reported. If this is more than 10p per day then we consider it to be worth shifting.

In [ ]:
# The simulation
def sim_trial_schedule(schedule_start_time, trial_start_time, duration, power):

    schedule = create_schedule(schedule_start_time, duration)
    trial_schedule = create_schedule(trial_start_time, duration)
    
    opt = pd.DataFrame(flex_pot['time']).reset_index()
    opt['tariff'] = flex_pot['2025-03-12']
    
    opt['schedule'] =  opt['time'].apply(lambda x: 1 if x in schedule else 0)
    opt['schedule_load_cost'] = opt['schedule']* power/2 * opt['tariff']
    
    opt['trial_schedule'] = opt['time'].apply(lambda x: 1 if x in trial_schedule else 0)
    opt['trial_load_cost'] = opt['trial_schedule']* power/2 * opt['tariff']
    
    opt['night'] = opt['time'].apply(lambda x: 1 if x < dt.time(6,0) else 1 if x > dt.time(23,0) else 0) # only works if schedule is during day
    opt['night_cost'] = opt['night'] * opt['trial_schedule'] * 50
    
    start_time_index = opt[opt['time']==schedule_start_time].index.item()
    opt['inconvenient'] = opt['index'].apply(lambda x: (start_time_index-x) if x < start_time_index else (x-start_time_index-duration*2+1)*2 if x >= start_time_index+duration*2 else 0)
    opt['inconvenient_cost'] = opt['trial_schedule'] * opt['inconvenient']
    
    opt['total_cost'] = opt['trial_load_cost'] + opt['night_cost'] + opt['inconvenient_cost']

    cost = opt['total_cost'].sum()
    true_cost = opt['trial_load_cost'].sum()
    
    return opt, cost, true_cost

In [ ]:
# Create a device on/off schedule based on the parameters specified
def create_schedule(schedule_start_time, duration):
    schedule = []
    schedule_start_date = dt.datetime.combine(dt.date.today(), schedule_start_time)
    for hh in range(duration*2):
        schedule_time = (schedule_start_date + dt.timedelta(minutes=30*hh)).time()
        schedule.append(schedule_time) 

    return schedule
    

In [ ]:
# Specify the load to be shifted

schedule_start_time = dt.time(17,0,0)
duration = 2 #hours   #NB doesn't work for 0.5 hour yet
power = 2 #kW


In [ ]:
# Run simulations to see if it is worth shifting the load

trial_cost=[]
true_trial_cost=[]
trial_start_times = pd.date_range(dt.datetime(2000,1,1,0,0,0),periods=48-duration*2+1,freq='30min')

for trial_start_time in trial_start_times:
    opt, cost, true_cost = sim_trial_schedule(schedule_start_time, trial_start_time.time(), duration, power)
    trial_cost.append(cost)
    true_trial_cost.append(true_cost)

In [ ]:
# Plot results 
results=pd.DataFrame(trial_start_times.time, trial_cost).reset_index()
results.columns=['trial_cost','trial_start_time']
results['trial_start_time'] = results['trial_start_time']
results['trial_true_cost'] = true_trial_cost
optimum_time = results[results['trial_cost']==results['trial_cost'].min()]['trial_start_time'].item()

results.plot('trial_start_time', ['trial_cost', 'trial_true_cost'], style='.-')
plt.axvline(schedule_start_time, color='black', linestyle='--', label='Scheduled time')
plt.axvline(optimum_time, color='black', linestyle='-', label='Optimum time')
plt.legend()

plt.tight_layout()

In [ ]:
# Calculate the cost savings between the schedule and optimum times
opt, cost, true_cost = sim_trial_schedule(schedule_start_time, optimum_time, duration, power)
cost_saving = (opt['schedule_load_cost']-opt['trial_load_cost']).sum()
total_saving = (opt['schedule_load_cost']-opt['total_cost']).sum()
print(schedule_start_time, optimum_time, cost_saving, total_saving)


In [ ]:
# Determine if there is another time which saves more than 10p and isn't too inconvenient
if (total_saving > 0) & (cost_saving > 10):
    print('You would save £%s by switching your schedule from %s to %s' % (round(cost_saving)/100, schedule_start_time, optimum_time)) # e.g. 5pm
elif (total_saving == 0):
    print('It is not worth switching your schedule today because the inconvenience is not worth the saving') # e.g. 12pm
else:
    print('It is not worth switching your schedule today because the cost saving is less than 10p') # e.g. 7am

In [ ]:
opt